In [125]:
import pyomo.environ as pyomo

# solver
solver = pyomo.SolverFactory('cbc',executable=r'C:\Cbc-master-win64-msvc16-mt\bin\cbc.exe')
solver.options['sec'] = 5

In [126]:
import pandas as pd 
df = pd.read_csv('pyomo_data_small_no_t.csv')

buyer 

In [127]:
# constraints 

#sum_{i,t}Y_{i,t}<= Q^F_i 
def request(model):
   return model.Y_it <= model.QF_i
#model.requestConstraint = pyomo.Constraint(rule=request)

#  f_{i,t}*g_{i,t} <= CO2_{i,t}
def carbon_target(model):  
    return model.f_it * model.g_it <= model.CO2_it
#model.carbontargetConstraint = pyomo.Constraint(rule=carbon_target)

# Y_{i,t} + g_{i,t} + Q^S_{i,t} = D_{i,t} 
def demand(model): 
    return (model.Y_it + model.g_it + model.QS_it == model.D_it) 
# model.demandConstraint = pyomo.Constraint(rule=demand)

#  g_{i,t} <= \sum^t_{t'=1} h_{i,t'}
def generator_capacity (model):  
    return model.g_it <= model.h_it
#model.generatorConstraint = pyomo.Constraint(rule=generator_capacity)


for i in range (len(df)): 
    # create a model 
    model = pyomo.ConcreteModel()

    # Parameters
    model.C_geninv_it = pyomo.Param(initialize=df['C_geninv_it'][i])  # C^{geninv}_{i,t}
    model.C_gen_it = pyomo.Param(initialize=df['C_gen_it'][i])        # C^{gen}_{i,t}
    model.f_it = pyomo.Param(initialize=df['f_it'][i])                # f_{i,t}
    model.CO2_it = pyomo.Param(initialize = df['CO2_it'][i])        #CO2_{i,t}
    model.D_it = pyomo.Param(initialize = df['D_it'][i])     #D_{i,t}
    model.PS_it = pyomo.Param(initialize = df['PS_it'][i])    # P^S_{i,t}
    model.PF_i = pyomo.Param(initialize=1)  


    # Variables
    model.g_it = pyomo.Var(domain=pyomo.NonNegativeReals)       # g_{i,t}
    model.h_it = pyomo.Var(domain=pyomo.NonNegativeReals)       # h_{i,t}
    model.QF_i = pyomo.Var(domain=pyomo.NonNegativeReals)       # Q^F_i
    model.QS_it = pyomo.Var(domain=pyomo.NonNegativeReals)      # Q^S_{i,t}
    model.Y_it = pyomo.Var(domain=pyomo.NonNegativeReals)     #Y_{i,t}

    # Objective Function
    #min P^F_i * Q^F_i + C^{geninv}_{i,t} * h_{i,t} + C^{gen}_{i,t} * g_{i,t} + P^S_{i,t} * Q^S_{i,t}
    def objective_func(model):
        return model.PF_i * model.QF_i \
                + model.C_geninv_it * model.h_it \
                + model.C_gen_it * model.g_it \
                + model.PS_it * model.QS_it 
    model.objective = pyomo.Objective(rule = objective_func, sense = pyomo.minimize)
    model.requestConstraint = pyomo.Constraint(rule=request)
    model.carbontargetConstraint = pyomo.Constraint(rule=carbon_target)
    model.demandConstraint = pyomo.Constraint(rule=demand)
    model.generatorConstraint = pyomo.Constraint(rule=generator_capacity)

    print(model.objective)

    result = solver.solve(model)

    print(result.solver.status)
    print(result.solver.termination_condition)

    print(model.display())
    

objective
ok
optimal
Model unknown

  Variables:
    g_it : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   5.0 :  None : False : False : NonNegativeReals
    h_it : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   5.0 :  None : False : False : NonNegativeReals
    QF_i : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    QS_it : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    Y_it : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals

  Objectives:
    objective : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :   0.1

  Cons